# Amini Cocoa Contamination Challenge Solution By SPECIALZ🔥🔥🙌🏽

This notebook documents my submission to the Amini Cocoa Contamination Challenge, which aims to build machine learning models capable of identifying multiple cocoa leaf diseases—like CSSVD and anthracnose—directly from images. The goal is to develop models that not only generalize well to unseen diseases, but also run efficiently on low-end smartphones, making real-time diagnosis accessible to smallholder farmers across Africa.

### ⚙️ Our Approach
Throughout this challenge, We experienced several ups and downs—experimenting with different YOLO models, adjusting image sizes, tuning confidence thresholds, and merging predictions. Getting a model that balanced accuracy, robustness, and efficiency was not easy. At one point, results improved significantly around the 46th epoch, showing the importance of careful monitoring and validation.

This notebook covers:

Model training using YOLOv11

Inference with test-time augmentation (TTA)

Prediction fusion using Weighted Box Fusion (WBF)

Final submission preparation

And an explainability section to meet the competition requirements

Let’s dive into the code and explore what worked—and what didn’t—on the path to building an AI solution that could one day live on a farmer’s phone.

### Install Necessary Packages

In [1]:
!pip -q install -U ultralytics iterative-stratification

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 81.6 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pylibcugraph-cu12 24.12.0 requires pylibraft-cu12==24.12.*, but you have pylibraft-cu12 25.2.0 which is incompatible.
pylibcugraph-cu12 24.12.0 requires rmm-cu12==24.12.*, but you have rmm-cu12 25.2.0 which is incompatible.


### Import Necessary Packages

In [2]:
import pandas as pd
import os
from pathlib import Path
import shutil
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
import cv2
import yaml
import matplotlib.pyplot as plt
from ultralytics import YOLO
import multiprocessing
import warnings
warnings.filterwarnings("ignore")
import random
from datetime import datetime
import time
from glob import glob
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from PIL import Image
import torch
import numpy as np
from ultralytics import RTDETR
from ultralytics.data.build import YOLODataset
import ultralytics.data.build as build
device='cuda'


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


### 📁  Set configurations and seed

In [3]:
class CFG:
    seed = 42
    random_state = 42
    folds=10

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.use_deterministic_algorithms(True, warn_only=True)
seed_everything(CFG.seed)

In [4]:
# Read the CSV File
df = pd.read_csv('/kaggle/input/amini-cocoa-contamination-challenge/Train.csv')

# Extract Unique Class Labels
unique_classes = df['class'].unique()

# Create a Class-to-Index Mapping
class_mapping = {cls: idx for idx, cls in enumerate(unique_classes)}
print(class_mapping)

{'healthy': 0, 'anthracnose': 1, 'cssvd': 2}


### ✅ Purpose
This is a common preprocessing step in machine learning tasks to convert categorical class labels into numerical values that can be used for model training.

### 📁 Directory and Data Loading

In [5]:
# Set the data directory
DATA_DIR = Path('/kaggle/input/amini-cocoa-contamination-challenge/')
IMGS_DIR = Path('/kaggle/input/amini-cocoa-contamination-challenge/dataset/images')

# Load train and test files
train = pd.read_csv(DATA_DIR / 'Train.csv')
test = pd.read_csv(DATA_DIR / 'Test.csv')
ss = pd.read_csv(DATA_DIR / 'SampleSubmission.csv')

### ✅ Purpose
This setup organizes paths and loads necessary CSV files for training, inference, and submission formatting in a Kaggle environment.

### 📁 Let's explore the Train, Test, Sample Submission File

In [6]:
train.head()

,Image_ID,class,confidence,ymin,xmin,ymax,xmax,class_id,ImagePath
0,ID_nBgcAR.jpg,healthy,1.0,75.0,15.0,162.0,195.0,2,dataset/images/train/ID_nBgcAR.jpg
1,ID_nBgcAR.jpg,healthy,1.0,58.0,1.0,133.0,171.0,2,dataset/images/train/ID_nBgcAR.jpg
2,ID_nBgcAR.jpg,healthy,1.0,42.0,29.0,377.0,349.0,2,dataset/images/train/ID_nBgcAR.jpg
3,ID_Kw2v8A.jpg,healthy,1.0,112.0,124.0,404.0,341.0,2,dataset/images/train/ID_Kw2v8A.jpg
4,ID_Kw2v8A.jpg,healthy,1.0,148.0,259.0,413.0,412.0,2,dataset/images/train/ID_Kw2v8A.jpg


In [7]:
test.head()

,Image_ID,class,confidence,ymin,xmin,ymax,xmax,class_id,ImagePath
0,ID_Genxyu.jpg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,dataset/images/test/ID_Genxyu.jpg
1,ID_svY6TG.jpg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,dataset/images/test/ID_svY6TG.jpg
2,ID_d0gpda.jpg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,dataset/images/test/ID_d0gpda.jpg
3,ID_frWmBT.jpg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,dataset/images/test/ID_frWmBT.jpg
4,ID_TaRW6o.jpg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,dataset/images/test/ID_TaRW6o.jpg


In [8]:
ss.head()

,Image_ID,class,confidence,ymin,xmin,ymax,xmax
0,ID_Genxyu.jpg,healthy,0.5,100,100,100,100
1,ID_svY6TG.jpg,healthy,0.5,100,100,100,100
2,ID_d0gpda.jpg,healthy,0.5,100,100,100,100
3,ID_frWmBT.jpg,healthy,0.5,100,100,100,100
4,ID_TaRW6o.jpg,healthy,0.5,100,100,100,100


Clearly, the test set does not have values for the bounding boxes and confidence which makes sense because we are to find them.

Let's explore furthermore

### 🏷️ Checking for Duplicates

In [9]:
# Defininig columns in the train set
defined_cols = ['Image_ID',	'confidence',	'class',	'ymin',	'xmin',	'ymax',	'xmax']
train = train[defined_cols]

print(f'Sum of duplicated colums: {train.duplicated().sum()}')
print(f'Size of dataframe before removing duplicates: {train.shape}')

# Remove duplicates
train = train.drop_duplicates()
print(f'Sum of duplicated colums after removing duplicates: {train.duplicated().sum()}')
print(f'Size of dataframe after removing duplicates: {train.shape}')

Sum of duplicated colums: 0
Size of dataframe before removing duplicates: (9792, 7)
Sum of duplicated colums after removing duplicates: 0
Size of dataframe after removing duplicates: (9792, 7)


### 🏷️ Generating Class Label Dictionary

In [10]:
unique_classes = train['class'].unique()
full_label_dict = {cls: idx for idx, cls in enumerate(unique_classes)}
full_label_dict

{'healthy': 0, 'anthracnose': 1, 'cssvd': 2}

### ✅ Purpose
Converts categorical class labels into numerical indices for use in model training and evaluation.

## 🧪 Data Preparation and Stratified K-Fold Setup for Multi-Label Classification

### Group Labels by Image

In [11]:
# Step 1: Group by Image_ID and aggregate class labels into lists
train['new_class'] = train['class'].map(full_label_dict)
grouped = train.groupby('Image_ID')['new_class'].apply(list).reset_index()

### ✅ Purpose
1. Maps class labels to integers using full_label_dict.
2. Groups the data by Image_ID, aggregating all associated class labels into lists.
3. This ensures each image has a list of all labels assigned to it (for multi-label classification).


### Initialize Class Columns with -1

In [12]:
all_classes = train["class"].unique().tolist()
for unique_class in all_classes:
    grouped[unique_class] = -1

### ✅ Purpose
1. Retrieves the list of all unique class names from the training data.

2. Adds a column for each class in the grouped DataFrame.

3. Initializes these columns to -1, which will later be updated to 1 or 0 based on label presence.

### Reverse Label Mapping

In [13]:
reverse_label_mapping = {full_label_dict[key]:key for key in full_label_dict} 

In [14]:
reverse_label_mapping

{0: 'healthy', 1: 'anthracnose', 2: 'cssvd'}

### ✅ Purpose
1. Creates a dictionary to map numeric labels back to their original string labels.

2. This is useful for labeling class columns correctly in the next step.

### One-Hot Encode Labels for Each Image

In [15]:
# input 1 if the label is in that image else 0
all_labels_list = (list(grouped['new_class'].values))
for train_index, label_List in enumerate(all_labels_list):
    unique_labels = list(set(label_List))
    for label_index in range(len(unique_labels)):
        label = int(unique_labels[label_index])
        for key_value in range(23):
            if label == key_value:
                grouped.loc[train_index, reverse_label_mapping[key_value]] = 1
                break

### ✅ Purpose
1. Iterates through each image's list of labels.

2. Converts the list into a set of unique labels.

3. For each label, checks if it matches a known label index.

If matched, sets the corresponding column in grouped to 1, indicating presence of that class label for that image.

### Create Stratified Folds and Image Paths

In [16]:
X = grouped[['Image_ID']]
grouped['fold'] = -1
mskf = MultilabelStratifiedKFold(n_splits=CFG.folds, shuffle=True, random_state=CFG.random_state)
for i_fold, (train_index, test_index) in enumerate(mskf.split(X, grouped[all_classes])):
    grouped.loc[test_index, "fold"] = i_fold     

### ✅ Purpose
1. Initializes a MultilabelStratifiedKFold for balanced fold generation.

2. Splits the data so that each fold has a similar distribution of labels across classes.

3. Assigns a fold number to each image for use in cross-validation.

In [17]:
# create image_path for grouped_data
grouped['image_path'] = [Path(str(IMGS_DIR) + '/train/' + x) for x in grouped.Image_ID]

# drop duplicates rows for test
test = test.drop_duplicates(subset=['Image_ID'], ignore_index=True)
test['image_path'] = [Path(str(IMGS_DIR) + '/test/' + x) for x in test.Image_ID]  

## 📝 Function to Convert Bounding Boxes to YOLO Format and Save Annotations

---

### 📦 Function: `save_yolo_annotation`

All values are normalized by the image width and height.

### Steps:

1. Reads the image to get its dimensions.

2. Normalizes bounding box coordinates.

3. Writes a .txt label file for the corresponding image in YOLO format.

4. Failsafe: If the image cannot be read, an error is raised.

In [18]:
# Function to convert the bounding boxes to YOLO format and save them
def save_yolo_annotation(row):

    image_path, class_id, output_dir = row['image_path'], row['class_id'], row['output_dir']

    img = cv2.imread(str(image_path))
    if img is None:
        raise ValueError(f"Could not read image from path: {image_path}")

    height, width, _ = img.shape
    label_file = Path(output_dir) / f"{Path(image_path).stem}.txt"


    ymin, xmin, ymax, xmax = row['ymin'], row['xmin'], row['ymax'], row['xmax']

    # Normalize the coordinates
    x_center = (xmin + xmax) / 2 / width
    y_center = (ymin + ymax) / 2 / height
    bbox_width = (xmax - xmin) / width
    bbox_height = (ymax - ymin) / height

    with open(label_file, 'a') as f:
        f.write(f"{class_id} {x_center:.6f} {y_center:.6f} {bbox_width:.6f} {bbox_height:.6f}\n")

### 🔍 Explanation
Purpose: Converts bounding boxes from (xmin, ymin, xmax, ymax) format to the YOLO format:
```arduino
class_id x_center y_center width height
```

In [19]:
# Parallelize the annotation saving process
def process_dataset(dataframe, output_dir):
    dataframe['output_dir'] = output_dir
    # convert the dataframe to a dictionary
    dataframe = dataframe.to_dict('records')
    for i in tqdm(range(len(dataframe))):
        save_yolo_annotation(dataframe[i])

### 🔍 Explanation
1. Adds an output_dir column to the DataFrame for storing annotation files.

2. Converts the DataFrame to a list of dictionaries (records) for easier iteration.

3. Iterates over each row and calls save_yolo_annotation to save YOLO labels.

4. A progress bar (tqdm) tracks the process for better visibility.

#### Let's Implement this now!

In [20]:
# # Add an image_path column
train['image_path'] = [Path(str(IMGS_DIR) + '/train/' + x) for x in train.Image_ID]

# Map string classes to integers (label encoding targets)
train['class_id'] = train['class'].map(full_label_dict)

## 🔁 Preparing YOLO Training/Validation/Test Data for a Specific Fold

This code processes one fold (specifically `fold == 1`) of your dataset to organize images and labels into the required format for training a YOLO model.

---

In [ ]:
# 🧠 Loop Through Folds

for fold in range(CFG.folds):
    if fold == 1:
        # 📁 Define Directory Structure
        # images
        TRAIN_IMAGES_DIR = Path(f'/kaggle/working/train/images/fold_{fold + 1}')
        VAL_IMAGES_DIR = Path(f'/kaggle/working/val/images/fold_{fold + 1}')
        TEST_IMAGES_DIR = Path('/kaggle/working/test/images')

        # labels
        TRAIN_LABELS_DIR = Path(f'/kaggle/working/train/labels/fold_{fold + 1}')
        VAL_LABELS_DIR = Path(f'/kaggle/working/val/labels/fold_{fold + 1}')
        TEST_LABELS_DIR = Path('/kaggle/working/test/labels')

        # Get the train and val for that fold
        train_fold = grouped[grouped['fold'] != fold ].reset_index(drop=True)
        val_fold = grouped[grouped['fold'] == fold].reset_index(drop=True)

        DIRS = [TRAIN_IMAGES_DIR, VAL_IMAGES_DIR, TRAIN_LABELS_DIR, VAL_LABELS_DIR, TEST_IMAGES_DIR, TEST_LABELS_DIR]
        
        # Create necessary directories
        for DIR in DIRS:
            if DIR.exists():
                shutil.rmtree(DIR)
            DIR.mkdir(parents=True, exist_ok=True)
       
        # Copy train, val, and test images to their respective dirs
        for img in tqdm(train_fold.image_path.unique()):
            shutil.copy(img, TRAIN_IMAGES_DIR / img.parts[-1])
        print(f'Copied train file for fold{fold+1} to folder')

        for img in tqdm(val_fold.image_path.unique()):
            shutil.copy(img, VAL_IMAGES_DIR / img.parts[-1])
        print(f'Copied val file for fold{fold+1} to folder')

        for img in tqdm(test.image_path.unique()):
            shutil.copy(img, TEST_IMAGES_DIR / img.parts[-1])
        print(f'Copied test file for first fold to folder')


        X_train = train[train.Image_ID.isin(train_fold.Image_ID)].reset_index(drop=True)
        X_val = train[train.Image_ID.isin(val_fold.Image_ID)].reset_index(drop=True)


        print(f"-------------Process Datasets for fold {fold+1}")
        # Save train and validation labels to their respective dirs
        process_dataset(X_train, TRAIN_LABELS_DIR)
        process_dataset(X_val, VAL_LABELS_DIR)

        print(f"-------------End of Processing of Datasets for fold {fold+1}")

  0%|          | 0/4976 [00:00<?, ?it/s]

Copied train file for fold2 to folder


  0%|          | 0/553 [00:00<?, ?it/s]

Copied val file for fold2 to folder


  0%|          | 0/1626 [00:00<?, ?it/s]

Copied test file for first fold to folder
-------------Process Datasets for fold 2


  0%|          | 0/8849 [00:00<?, ?it/s]

  0%|          | 0/943 [00:00<?, ?it/s]

### ✅ Summary
This fold-wise script performs the following:

Sets up directories for training, validation, and test images and labels.

Splits the dataset by fold.

Copies image files into YOLO-style directories.

Converts and saves labels in YOLO format using the process_dataset function.

Prepares data for training object detection models like YOLOv5 or YOLOv8.

ℹ️ Only fold == 1 is processed in this script, which can be extended to process all folds by removing the if fold == 1 condition.

### This is Optional. If you are using Kaggle, run this cell to avoid getting errors

In [25]:
# Define the new dataset directory structure within the current working directory
base_dir = './datasets'  # Create the dataset in the local writable directory
dirs = [
    os.path.join(base_dir, 'train/images'),
    os.path.join(base_dir, 'train/labels'),
    os.path.join(base_dir, 'val/images'),
    os.path.join(base_dir, 'val/labels')
]

# Create the directories
for dir_path in dirs:
    os.makedirs(dir_path, exist_ok=True)

# Example: Source directories where your current files are stored (update these paths)
source_train_images = './train/images'
source_train_labels = './train/labels'
source_val_images = './val/images'
source_val_labels = './val/labels'

# Move files to the new structure
def move_files(source, destination):
    if os.path.exists(source):
        for file_name in os.listdir(source):
            shutil.move(os.path.join(source, file_name), destination)

# Move training images and labels
move_files(source_train_images, os.path.join(base_dir, 'train/images'))
move_files(source_train_labels, os.path.join(base_dir, 'train/labels'))

# Move validation images and labels
move_files(source_val_images, os.path.join(base_dir, 'val/images'))
move_files(source_val_labels, os.path.join(base_dir, 'val/labels'))

## 📄 Create `data.yaml` File for YOLO Training

This script creates the `data.yaml` file, which is required by YOLO models (such as YOLOv11 or YOLOv12) to define the training configuration.

In [26]:
# Create a data.yaml file required by YOLO
# 🏷️ Extract Class Names and Count
class_names = train['class'].unique().tolist()
num_classes = len(class_names)

# 📁 Define Training and Validation Image Directories
# images
TRAIN_IMAGES_DIR = Path('/kaggle/working/datasets/train/images/fold_2/')
VAL_IMAGES_DIR = Path('/kaggle/working/datasets/val/images/fold_2/')

#📘 Build YAML Configuration Dictionary
data_yaml = {
    'train': str(TRAIN_IMAGES_DIR),
    'val': str(VAL_IMAGES_DIR),
    'nc': num_classes,
    'names': class_names
}

# Save the data.yaml file
yaml_path = Path('data.yaml')
with open(yaml_path, 'w') as file:
    yaml.dump(data_yaml, file, default_flow_style=False)

### ✅ Purpose
Constructs the dictionary for the data.yaml file, containing:

train: path to training images.

val: path to validation images.

nc: number of classes.

names: list of class names (in order).

Saves the constructed dictionary into a data.yaml file.

This file will be used by YOLO during training to understand dataset structure and class names.

In [27]:
data_yaml

{'train': '/kaggle/working/datasets/train/images/fold_2',
 'val': '/kaggle/working/datasets/val/images/fold_2',
 'nc': 3,
 'names': ['healthy', 'anthracnose', 'cssvd']}

## 🧠 Train First Model, A YOLOv11 Model on Custom Dataset

This code snippet initializes a YOLOv11 model and trains it using the configuration specified in the `data.yaml` file.


### 🚀 Hyperparameters
```yaml
data: Path to the data.yaml file describing the dataset.

epochs: Train for 100 full passes over the training dataset.

imgsz: Resize input images to 576×576 pixels.

device: Use GPU device 0 for training.

batch: Batch size of 16 images per step.

optimizer: Use the AdamW optimizer (better regularization).

lr0: Initial learning rate of 3e-4.

momentum: Momentum factor for optimizer (used if switching to SGD).

weight_decay: L2 regularization to prevent overfitting.

close_mosaic: Disable YOLO’s mosaic augmentation after 30 epochs (stabilizes training).

seed: Set a fixed random seed (42) for reproducibility.

patience: Set to stop the training after no improvement
```

In [ ]:
# 🔧 Load Pretrained YOLOv11 Model
model1 = YOLO("yolo11l.pt")

# 🚀 Simulate 100-Epoch Training in 46 Epochs
model1.train(data='data.yaml',
             epochs=100,               
             imgsz=640,
             device=0,
             batch=16,
             optimizer='AdamW',
             lr0=3e-4,           
             momentum=0.9,
             weight_decay=1e-2,
             close_mosaic=30,
             seed=42,
             patience=10      
)

### 📈 Outcome
This setup fine-tunes the YOLOv11 large model on your custom dataset, leveraging strong regularization and augmentation settings to improve generalization. After training, the best model checkpoint will be saved and can be used for inference or further evaluation.

## ✅ Evaluate the Trained YOLOv11 Model

After training, we evaluate the model's performance on the validation dataset using the `.val()` method.

In [28]:
results = model1.val()

Ultralytics 8.3.133 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11l summary (fused): 190 layers, 25,281,625 parameters, 0 gradients, 86.6 GFLOPs


100%|██████████| 755k/755k [00:00<00:00, 18.0MB/s]

val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2471.5±1298.1 MB/s, size: 1661.7 KB)



val: Scanning /kaggle/working/datasets/val/labels/fold_2... 553 images, 0 backgrounds, 0 corrupt: 100%|██████████| 553/553 [00:03<00:00, 169.13it/s]

val: /kaggle/working/datasets/val/images/fold_2/ID_BFveJq.jpeg: corrupt JPEG restored and saved
val: /kaggle/working/datasets/val/images/fold_2/ID_By57N4.jpeg: corrupt JPEG restored and saved
val: /kaggle/working/datasets/val/images/fold_2/ID_EJWqGf.jpeg: corrupt JPEG restored and saved
val: /kaggle/working/datasets/val/images/fold_2/ID_EUJ6CX.jpeg: corrupt JPEG restored and saved
val: /kaggle/working/datasets/val/images/fold_2/ID_FtYRqz.jpeg: corrupt JPEG restored and saved
val: /kaggle/working/datasets/val/images/fold_2/ID_HuMwmi.jpg: corrupt JPEG restored and saved
val: /kaggle/working/datasets/val/images/fold_2/ID_J7hL2Y.jpg: corrupt JPEG restored and saved
val: /kaggle/working/datasets/val/images/fold_2/ID_NGiOVN.jpg: corrupt JPEG restored and saved
val: /kaggle/working/datasets/val/images/fold_2/ID_SuY5t1.jpeg: corrupt JPEG restored and saved
val: /kaggle/working/datasets/val/images/fold_2/ID_YevJzo.jpeg: corrupt JPEG restored and saved
val: /kaggle/working/datasets/val/images/fo


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 35/35 [00:18<00:00,  1.87it/s]


                   all        553        943      0.788      0.751      0.821      0.571
               healthy        172        386      0.744      0.731      0.801      0.549
           anthracnose        136        194      0.808      0.803      0.862      0.612
                 cssvd        246        363      0.811      0.719        0.8      0.553
Speed: 0.2ms preprocess, 22.9ms inference, 0.0ms loss, 1.8ms postprocess per image
Results saved to runs/detect/val


Our Cross Validation is at 0.821 which is very great. Let's train another model and ensemble it.

Save best.pt for inference

## 🧠 Train Second Model, A YOLOv11 Model on Custom YoloWeightedDataset
This code snippet initializes a YOLOv11 model and trains it using the configuration specified in the data.yaml file. A Yolo weighted dataset is utilized.

The YOLOWeightedDataset class tries to balance the custom for dataset training the model.

I followed a similar training pipeline to the one used in the [Ghana Crop Disease Detection Challenge Notebook by Raphael Kiminya](https://github.com/kiminya-raphael/Ghana-Crop-Disease-Detection-challenge/blob/main/1_train.ipynb), making modifications specific to this dataset and task.


In [ ]:
class YOLOWeightedDataset(YOLODataset):
    def __init__(self, *args, mode="train", **kwargs):
        """
        Initialize the WeightedDataset.

        Args:
            class_weights (list or numpy array): A list or array of weights corresponding to each class.
        """

        super(YOLOWeightedDataset, self).__init__(*args, **kwargs)

        self.train_mode = "train" in self.prefix

        # You can also specify weights manually instead
        self.count_instances()
        class_weights = np.sum(self.counts) / self.counts

        # Aggregation function
        self.agg_func = np.mean

        self.class_weights = np.array(class_weights)
        self.weights = self.calculate_weights()
        self.probabilities = self.calculate_probabilities()

    def count_instances(self):
        """
        Count the number of instances per class

        Returns:
            dict: A dict containing the counts for each class.
        """
        self.counts = [0 for i in range(len(self.data["names"]))]
        for label in self.labels:
            cls = label['cls'].reshape(-1).astype(int)
            for id in cls:
                self.counts[id] += 1

        self.counts = np.array(self.counts)
        self.counts = np.where(self.counts == 0, 1, self.counts)

    def calculate_weights(self):
        """
        Calculate the aggregated weight for each label based on class weights.

        Returns:
            list: A list of aggregated weights corresponding to each label.
        """
        weights = []
        for label in self.labels:
            cls = label['cls'].reshape(-1).astype(int)

            # Give a default weight to background class
            if cls.size == 0:
              weights.append(1)
              continue

            # Take mean of weights
            # You can change this weight aggregation function to aggregate weights differently
            weight = self.agg_func(self.class_weights[cls])
            weights.append(weight)
        return weights

    def calculate_probabilities(self):
        """
        Calculate and store the sampling probabilities based on the weights.

        Returns:
            list: A list of sampling probabilities corresponding to each label.
        """
        total_weight = sum(self.weights)
        probabilities = [w / total_weight for w in self.weights]
        return probabilities

    def __getitem__(self, index):
        """
        Return transformed label information based on the sampled index.
        """
        # Don't use for validation
        if not self.train_mode:
            return self.transforms(self.get_image_and_label(index))
        else:
            index = np.random.choice(len(self.labels), p=self.probabilities)
            return self.transforms(self.get_image_and_label(index))

build.YOLODataset = YOLOWeightedDataset

Just like the first model, we used the same model with its respective hyperparameters

In [ ]:
# 🔧 Load Pretrained YOLOv11 Model
model2 = YOLO("yolo11l.pt")

# 🚀 Simulate 100-Epoch Training in 46 Epochs
model2.train(data='data.yaml',
             epochs=100,               
             imgsz=640,
             device=0,
             batch=16,
             optimizer='AdamW',
             lr0=3e-4,           
             momentum=0.9,
             weight_decay=1e-2,
             close_mosaic=30,
             seed=42,
             patience=10      
)

After training, we evaluate the model's performance on the validation dataset using the `.val()` method.

In [29]:
results = model2.val()

Ultralytics 8.3.133 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11l summary (fused): 190 layers, 25,281,625 parameters, 0 gradients, 86.6 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2015.1±557.3 MB/s, size: 728.6 KB)


val: Scanning /kaggle/working/datasets/val/labels/fold_2.cache... 553 images, 0 backgrounds, 0 corrupt: 100%|██████████| 553/553 [00:00<?, ?it/s]

val: /kaggle/working/datasets/val/images/fold_2/ID_BFveJq.jpeg: corrupt JPEG restored and saved
val: /kaggle/working/datasets/val/images/fold_2/ID_By57N4.jpeg: corrupt JPEG restored and saved
val: /kaggle/working/datasets/val/images/fold_2/ID_EJWqGf.jpeg: corrupt JPEG restored and saved
val: /kaggle/working/datasets/val/images/fold_2/ID_EUJ6CX.jpeg: corrupt JPEG restored and saved
val: /kaggle/working/datasets/val/images/fold_2/ID_FtYRqz.jpeg: corrupt JPEG restored and saved
val: /kaggle/working/datasets/val/images/fold_2/ID_HuMwmi.jpg: corrupt JPEG restored and saved
val: /kaggle/working/datasets/val/images/fold_2/ID_J7hL2Y.jpg: corrupt JPEG restored and saved
val: /kaggle/working/datasets/val/images/fold_2/ID_NGiOVN.jpg: corrupt JPEG restored and saved
val: /kaggle/working/datasets/val/images/fold_2/ID_SuY5t1.jpeg: corrupt JPEG restored and saved
val: /kaggle/working/datasets/val/images/fold_2/ID_YevJzo.jpeg: corrupt JPEG restored and saved
val: /kaggle/working/datasets/val/images/fo


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 35/35 [00:17<00:00,  1.96it/s]


                   all        553        943      0.791      0.736      0.818      0.571
               healthy        172        386      0.768      0.725      0.808      0.551
           anthracnose        136        194      0.785      0.784      0.852      0.613
                 cssvd        246        363      0.819        0.7      0.793      0.548
Speed: 0.2ms preprocess, 23.7ms inference, 0.0ms loss, 1.4ms postprocess per image
Results saved to runs/detect/val2


Our Cross Validation is at 0.818 which is slightly lower than the first model but is great. Let's now infer the models

Make sure to save both models for the inference in the next notebook

We are so sorry we could not show the train logs for the code review. We were short on time.
This training time is slightly below the 9 hour training time.

### Complete this notebook with the inference and explainability notebook